# Introduction

This notebook is a demonstration of how to use the general transfer matrix model by Rathmann et al. (2021) for modelling radar returns over ice columns with arbitrary crystal orientation fabric profiles.

The model consists of vertically-stacked layers of anisotropic polycrystalline ice. Each layer is characterized by its bulk dielectric permittivity tensor
$$ 
\langle\boldsymbol{\epsilon}_{\mathrm{g}}\rangle = \frac{2\epsilon_\perp+\epsilon_\parallel}{3}\mathbf{I} 
+ (\epsilon_\parallel-\epsilon_\perp) \left(\langle\mathbf{c}\otimes\mathbf{c}\rangle - \frac{1}{3}\mathbf{I}\right), 
$$
where $\epsilon_\parallel$ and $\epsilon_\perp$ are the single-grain principal permittivities parallel and perpendicular to the grain optical $\mathbf{c}$-axis, and $\langle\mathbf{c}\otimes\mathbf{c}\rangle$ is the second-order structure tensor (a.k.a. orientation tensor, $\mathbf{a}^{(2)}$). 

As a first step, the vertical profile of $\langle\mathbf{c}\otimes\mathbf{c}\rangle$ must be specified.
To get going, try out the below non-symmetric vertical single-maximum profile.

In [ ]:
import numpy as np
import sys
sys.path.insert(0, '../../lib') # Add model library path
from layerstack import c2_to_nlm

### Construct structure tensor profile

H        = 1500 # Ice column thickness
N_layers = 100  # Number of layers (for radar model below)

z   = np.linspace(0,-H, N_layers) # Interface positions
c2  = np.zeros((N_layers,3,3))    # Structure tensor. Note equivelent notations: c2 = <c^2> = a^(2)
nlm = np.zeros((N_layers,6), dtype=np.complex128) # Spectral ODF coefficients corresponding to <c^2>

lambdavec = np.linspace(1.5/3, 0.8, N_layers) # Principal eigen value (lambda_z) with depth
deltavec  = np.linspace(0, 0.1, N_layers)     # Horizontal asymmetry
c2[:,0,0] = 5/10*(1-lambdavec) + deltavec 
c2[:,1,1] = 5/10*(1-lambdavec) - deltavec
c2[:,2,2] = lambdavec

for ii in np.arange(N_layers):
    nlm[ii,:], lm = c2_to_nlm(c2[ii,:,:]) # Convert <c^2> to spectral coefficients n_2^m

nlm[0, 1:] = 0 # Ensure the top layer is isotropic (required by model)

### Plot ODFs

from plottools import plot_ODF

for I in [1, int(N_layers/2), -1]:
    plot_ODF(nlm[I,:], lm, tickintvl=4, cblabel=r'$\psi(z=\SI{%.1f}{\kilo\metre})$'%(z[I]*1e-3));


# Calculate radar returns

To calculate the radar returns 

* $\bar{P}_{jk}$          : Mean return power for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\delta{P}_{jk}$        : Angular power anomaly for Tx and Rx polarizations of $j=\lbrace \mathrm{H,V}\rbrace$ and $k=\lbrace \mathrm{H,V}\rbrace$, respectively
* $\varphi_{\mathrm{HV}}$ : HH$-$VV Covariance phase angle

given the above fabric profile, run:


In [ ]:
import sys
sys.path.insert(0, '../../lib') # Add model library path
from layerstack import *
from plottools import *

### Setup layer stack

N_frames = 50           # Number of horizontally rotated frames (beta) between 0 and Pi 
epsc     = 3.17         # Single grain permittivity parallel to c-axis
epsa     = epsc - 0.034 # Single grain permittivity perpendicular to c-axis
sigma    = 1e-5         # Isotropic macroscopic conductivity

lstack = LayerStack(nlm, z, N_frames=N_frames, epsa=[epsa], epsc=[epsc], sigma=[sigma]) # Assumes constant epsa, epsc and sigma throughout the column if len()==1

### Transmitted plane wave

E0       = 1e3   # Tx E-field amplitude
f        = 179e6 # Tx frequency in Hz
alpha    = np.deg2rad(0) # Angle of incidence in radians (0 = normal incidence)

returns = lstack.get_returns(E0, f=f, alpha=alpha) # returns = (Pm_HH,Pm_HV, dP_HH,dP_HV, c_HHVV, E_HH,E_HV)

### Plot results

(eigvals, e1,e2,e3, a2) = lstack.get_eigenbasis()

(plt, *_) = plot_returns(lstack.z, returns, a2, eigvals);
plt.show()